In [1]:
from importlib.metadata import version
import os

try:
    import xradio

    print("XRADIO version", version("xradio"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing XRADIO")

    os.system("pip install xradio")

    import xradio

    print("xradio version", version("xradio"), " installed.")


# These are the three API functions that are currently supported
from xradio.image import load_image, read_image, write_image

XRADIO version 0.0.25 already installed.


## Download dataset

https://open-bitbucket.nrao.edu/projects/CASA/repos/casatestdata/browse/unittest/tclean/venus_ephem_test.ms

In [2]:
import graphviper

graphviper.utils.data.download(file="sdimaging.ms")

[2024-04-11 10:38:26,490]     INFO  graphviper:  Couldn't find file metadata locally in /Users/jsteeb/Dropbox/graphviper/src/graphviper/utils/data/.dropbox/file.download.json, checking remote API ... 


sdimaging.ms.zip:   0%|          | 0.00/28.3M [00:00<?, ?iB/s]

[2024-04-11 10:38:28,841]     INFO  graphviper:  Extracting file: sdimaging.ms.zip ... 


## Convert to Processing Set

In [3]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

ms_file = "sdimaging.ms"
partition_scheme = "ddi_intent_field"
# partition_scheme="ddi_state"

main_chunksize = {"time": 63}  # baseline, polarization
outfile = "sdimaging.vis.zarr"
convert_msv2_to_processing_set(
    in_file=ms_file,
    out_file=outfile,
    partition_scheme=partition_scheme,
    parallel=False,
    overwrite=True,
    main_chunksize=main_chunksize,
)

## Processing Set

In [4]:
from xradio.vis.read_processing_set import read_processing_set

ps = read_processing_set(ps_store=outfile)
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency,shape,field_coords
0,sdimaging_ddi_0_intent_OBSERVE_TARGET_ON_SOURC...,0,"OBSERVE_TARGET_ON_SOURCE,FREQUENCY_SWITCH_SIG",0,FLS3a,1.419395e+09,1.421893e+09,"(3843, 1, 1024, 2)","[fk5, 0h00m00s, 0d00m00s]"


In [5]:
ps.keys()

dict_keys(['sdimaging_ddi_0_intent_OBSERVE_TARGET_ON_SOURCE,FREQUENCY_SWITCH_SIG_field_id_0'])

In [6]:
ps["sdimaging_ddi_0_intent_OBSERVE_TARGET_ON_SOURCE,FREQUENCY_SWITCH_SIG_field_id_0"]

<xarray.Dataset> Size: 71MB
Dimensions:                     (time: 3843, baseline_id: 1, frequency: 1024,
                                 polarization: 2, uvw_label: 3)
Coordinates:
    baseline_antenna1_id        (baseline_id) int32 4B dask.array<chunksize=(1,), meta=np.ndarray>
    baseline_antenna2_id        (baseline_id) int32 4B dask.array<chunksize=(1,), meta=np.ndarray>
  * baseline_id                 (baseline_id) int64 8B 0
  * frequency                   (frequency) float64 8kB 1.419e+09 ... 1.422e+09
  * polarization                (polarization) <U2 16B 'XX' 'YY'
  * time                        (time) float64 31kB 1.069e+09 ... 1.069e+09
  * uvw_label                   (uvw_label) <U1 12B 'u' 'v' 'w'
Data variables:
    EFFECTIVE_INTEGRATION_TIME  (time, baseline_id) float64 31kB dask.array<chunksize=(63, 1), meta=np.ndarray>
    FLAG                        (time, baseline_id, frequency, polarization) bool 8MB dask.array<chunksize=(63, 1, 1024, 2), meta=np.ndarray>
    SPECTRUM                    (time, baseline_id, frequency, polarization) float32 31MB dask.array<chunksize=(63, 1, 1024, 2), meta=np.ndarray>
    TIME_CENTROID               (time, baseline_id) float64 31kB dask.array<chunksize=(63, 1), meta=np.ndarray>
    UVW                         (time, baseline_id, uvw_label) float64 92kB dask.array<chunksize=(63, 1, 3), meta=np.ndarray>
    WEIGHT                      (time, baseline_id, frequency, polarization) float32 31MB dask.array<chunksize=(63, 1, 1024, 2), meta=np.ndarray>
Attributes:
    data_groups:   {'base': {'flag': 'FLAG', 'spectrum': 'SPECTRUM', 'uvw': '...
    ddi:           0
    intent:        OBSERVE_TARGET_ON_SOURCE,FREQUENCY_SWITCH_SIG
    antenna_xds:   <xarray.Dataset> Size: 152B\nDimensions:        (antenna_i...
    weather_xds:   <xarray.Dataset> Size: 56B\nDimensions:         (station_i...
    pointing_xds:  <xarray.Dataset> Size: 154kB\nDimensions:        (time: 38...

In [7]:
ps[
    "sdimaging_ddi_0_intent_OBSERVE_TARGET_ON_SOURCE,FREQUENCY_SWITCH_SIG_field_id_0"
].attrs["antenna_xds"]

<xarray.Dataset> Size: 152B
Dimensions:        (antenna_id: 1, xyz_label: 3)
Coordinates:
  * antenna_id     (antenna_id) int64 8B 0
    mount          (antenna_id) <U6 24B dask.array<chunksize=(1,), meta=np.ndarray>
    name           (antenna_id) <U3 12B dask.array<chunksize=(1,), meta=np.ndarray>
    station        (antenna_id) <U9 36B dask.array<chunksize=(1,), meta=np.ndarray>
    type           (antenna_id) <U1 4B dask.array<chunksize=(1,), meta=np.ndarray>
  * xyz_label      (xyz_label) <U1 12B 'x' 'y' 'z'
Data variables:
    DISH_DIAMETER  (antenna_id) float64 8B dask.array<chunksize=(1,), meta=np.ndarray>
    FEED_OFFSET    (antenna_id, xyz_label) float64 24B dask.array<chunksize=(1, 3), meta=np.ndarray>
    POSITION       (antenna_id, xyz_label) float64 24B dask.array<chunksize=(1, 3), meta=np.ndarray>